In [1]:
from simglucose.simulation.env import T1DSimEnv
from simglucose.controller.basal_bolus_ctrller import BBController
from simglucose.sensor.cgm import CGMSensor
from simglucose.actuator.pump import InsulinPump
from simglucose.patient.t1dpatient import T1DPatient
from simglucose.simulation.scenario_gen import RandomScenario
from simglucose.simulation.scenario import CustomScenario
from simglucose.simulation.sim_engine import SimObj, sim, batch_sim
from datetime import timedelta
from datetime import time as dt_time
from datetime import datetime

import matplotlib.pyplot as plt
    
import os
import numpy as np
import pandas as pd

In [3]:
# specify start_time as the beginning of today
now = datetime.now()
t0 = datetime.combine(now.date(), dt_time(6, 0, 0, 0))
start_time=t0
max_days=15
#start_time = datetime.combine(now.date(), datetime.min.time())

# --------- Create Random Scenario --------------
# Specify results saving path
path = './results'

df=pd.read_csv('vpatient_params.csv')
Npat=len(df)

# SIMULATE A POPULATION

## 1. Go through the list of available patients
## 2. For each patient generate 10 different patients:
### 2a. Setup patient loading its parameters from file
### 2b. Provide a small variation to its Vmx value to increase variability
### 2c. Setup CGM sensor and pump models
### 2d. Setup meal pattern (custom or randomized)
### 2d. Simulate the CGM measures

In [8]:
from simglucose.simulation.scenario_gen import RandomScenario

N_SAMPLE=288

new_index=0

Vmx=[];
CGM=np.empty((0,288));
CHO=np.empty((0,288));
insulin=np.empty((0,288));
filename=[];

for pat in range(Npat):
    patname=df.iloc[pat]['Name']

    for rep in range(10):
        ##################################################################################################################
        # 1. Setup patient loading its parameters from file
        ##################################################################################################################
        # Create a patient
        patient = T1DPatient.withName(patname)
        
        ext=1
        count=0
        while(ext>0 & count<10):
            #add random variation to the Vmx value to simulate more patients
            patient._params.Vmx=patient._params.Vmx+0.003*np.random.randn()
            patient_name='{:s}_{:02d}'.format(patname,rep) 

            print('Simulating: {:s}, repetition: {:d}'.format(patient_name,count))


            ##################################################################################################################
            # 2. Setup CGM sensor and pump models
            ##################################################################################################################
            # Creat sensor and insulin pump
            sensor = CGMSensor.withName('Dexcom', seed=1)
            pump = InsulinPump.withName('Insulet')

            ##################################################################################################################
            # 3. Setup custom meal pattern 
            # custom scenario is a list of tuples (time, meal_size)
            ##################################################################################################################
            #sim_time = timedelta(days=max_days)
            #now = datetime.now()
            #scenario=RandomScenario(start_time=t0, seed=42)

            ##################################################################################################################
            # 3. Setup custom meal pattern 
            # custom scenario is a list of tuples (time, meal_size)
            ##################################################################################################################
            prob = [0.95, 0.3, 0.95, 0.3, 0.95, 0.3]
            base_t=[7, 10, 12.5, 16, 20, 22]
            base_m = [10, 3, 20, 3, 20, 3]
            sigma_m = [2, 1, 3, 1, 3, 1]
            scen=[]

            for d in range(max_days):
                for i in range(6):
                    pmeal=np.random.uniform()
                    if(pmeal<prob[i]):
                        tm=(base_t[i]+(np.random.uniform()-0.5)*2)*60+d*24*60
                        vm=max(0,base_m[i]+sigma_m[i]*np.random.randn())
                        scen.append((tm,vm))

            scenario = CustomScenario(start_time=start_time, scenario=scen)

            ##################################################################################################################
            # 4. Simulate the CGM measures
            ##################################################################################################################

            env = T1DSimEnv(patient, sensor, pump, scenario)
            env.patient.name=patient_name

            # Create a controller
            controller = BBController()

            # Put them together to create a simulation object
            s2 = SimObj(env, controller, timedelta(days=max_days), animate=False, path=path)
            #print(env.patient.name)
            s2.simulate()
            results2=s2.env.show_history()

            cgm=results2.CGM
            ext=np.sum(cgm<=40)
        
            if(ext==0):
                for n_days in range(10):

                    patient_name='{:s}_{:02d}_day{:02d}'.format(patname,rep,n_days)

                    start_ind=(n_days+3)*N_SAMPLE
                    stop_ind=(n_days+3+1)*N_SAMPLE

                    Vmx.append(patient._params.Vmx)
                    filename.append(patient_name)

                    day_results=results2.iloc[start_ind:stop_ind]
                    savename = os.path.join(path, str(patient_name) + '.csv')
                    day_results.to_csv(savename)

                    CGM=np.vstack((CGM,day_results.CGM))
                    CHO=np.vstack((CHO,day_results.CGM))
                    insulin=np.vstack((insulin,day_results.insulin-day_results.insulin[0]))

                    #plt.plot(results2.CGM)
            
            count=count+1
            
            if count>10:
                break

Simulating: adolescent#001_00, repetition: 0
Simulating: adolescent#001_01, repetition: 0
Simulating: adolescent#001_02, repetition: 0
Simulating: adolescent#001_03, repetition: 0
Simulating: adolescent#001_04, repetition: 0
Simulating: adolescent#001_05, repetition: 0
Simulating: adolescent#001_06, repetition: 0
Simulating: adolescent#001_07, repetition: 0
Simulating: adolescent#001_08, repetition: 0
Simulating: adolescent#001_09, repetition: 0
Simulating: adolescent#002_00, repetition: 0
Simulating: adolescent#002_01, repetition: 0
Simulating: adolescent#002_02, repetition: 0
Simulating: adolescent#002_03, repetition: 0
Simulating: adolescent#002_04, repetition: 0
Simulating: adolescent#002_05, repetition: 0
Simulating: adolescent#002_06, repetition: 0
Simulating: adolescent#002_07, repetition: 0
Simulating: adolescent#002_08, repetition: 0
Simulating: adolescent#002_09, repetition: 0
Simulating: adolescent#003_00, repetition: 0
Simulating: adolescent#003_01, repetition: 0
Simulating

In [ ]:
 ext=np.sum(cgm<=40)

In [ ]:
simpar=pd.DataFrame()
simpar['Filename']=filename
simpar['Vmx']=Vmx

simpar.to_csv("Population_parameters.xlsx") 

In [ ]:
np.save('CGM_pop.np', CGM)
np.save('CHO_pop.np', CHO)
np.save('insulin_pop.np', insulin)


In [ ]:
np.save('Vmx_pop.np', Vmx)

In [ ]:
pat=0
rep=0
patname=df.iloc[pat]['Name']

##################################################################################################################
# 1. Setup patient loading its parameters from file
##################################################################################################################
# Create a patient
patient = T1DPatient.withName(patname)

#add random variation to the Vmx value to simulate more patients
patient._params.Vmx=patient._params.Vmx+0.003*np.random.randn()
patient_name='{:s}_{:02d}'.format(patname,rep) 


##################################################################################################################
# 2. Setup CGM sensor and pump models
##################################################################################################################
# Creat sensor and insulin pump
sensor = CGMSensor.withName('Dexcom', seed=1)
pump = InsulinPump.withName('Insulet')

##################################################################################################################
# 3. Setup custom meal pattern 
# custom scenario is a list of tuples (time, meal_size)
##################################################################################################################

from simglucose.simulation.scenario_gen import RandomScenario
sim_time = timedelta(days=max_days)
now = datetime.now()

scenario=RandomScenario(start_time=t0, seed=42)

##################################################################################################################
# 4. Simulate the CGM measures
##################################################################################################################

env = T1DSimEnv(patient, sensor, pump, scenario)
env.patient.name=patient_name

# Create a controller
controller = BBController()

# Put them together to create a simulation object
s2 = SimObj(env, controller, timedelta(days=max_days), animate=False, path=path)
print(env.patient.name)
s2.simulate()
results2=s2.env.show_history()

In [ ]:
results2

In [ ]:
print(results2.head())
dres=results2.iloc[289:2*289]
print(dres.head())

In [ ]:
results2.iloc[13*288]